In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
def direction(val):
    val1 = Conversion().convert_int256_bytes_to_int(val, signed=False) 
    val2 = Conversion().convert_int256_bytes_to_int(val, signed=True)
    return val1 != val2

In [3]:
chain = Net.POLYGON
platform = Platform.UNIV3
contract = JSONContract.UniswapV3Pool

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [4]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
mint_filt = Filter.create_filter(address=None, event_types=[Pair.events.Mint])  # Listen events from any smart contract

In [5]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 10

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=mint_filt)

In [8]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    event_signature, owner, tick_lower, tick_upper = evt["topics"]
    sender, liquidity_amount, amount0, amount1  = Conversion().decode_data(evt["data"])
    args = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['pool_address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['owner'] = Conversion().convert_uint256_hex_string_to_address(owner)
    event['details']['tick_lower'] = Conversion().convert_uint256_string_to_int(tick_lower, signed=True)
    event['details']['tick_upper'] = Conversion().convert_uint256_string_to_int(tick_upper, signed=True)  
    if(len(args) == 4):
        event['details']['liquidity_amount'] = Conversion().convert_int256_bytes_to_int(args[1])
        event['details']['amount0'] = Conversion().convert_int256_bytes_to_int(args[2])
        event['details']['amount1'] = Conversion().convert_int256_bytes_to_int(args[3])
    events[k] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")

Swap at block:60,352,427 tx:0x8c6808bcbcc411f34de65d49e930f14cfaef0b522f00cc1f93a46d7ccaacd8e6
.


In [9]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,pool_address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv3pool,mint,uniswap_v3,0x88eb11f4b1b4781b4471b9fa36e4f017d8d900a8,0x8c6808bcbcc411f34de65d49e930f14cfaef0b522f00...,60352427,1723147609,{'web3_type': <class 'web3._utils.datatypes.Mi...


In [10]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv3pool',
  'type': 'mint',
  'platform': 'uniswap_v3',
  'pool_address': '0x88eb11f4b1b4781b4471b9fa36e4f017d8d900a8',
  'tx_hash': '0x8c6808bcbcc411f34de65d49e930f14cfaef0b522f00cc1f93a46d7ccaacd8e6',
  'blk_num': 60352427,
  'timestamp': 1723147609,
  'details': {'web3_type': web3._utils.datatypes.Mint,
   'owner': '0xC36442b4a4522E871399CD717aBDD847Ab11FE88',
   'tick_lower': -211200,
   'tick_upper': -189200,
   'liquidity_amount': 174301018756543,
   'amount0': 1671680363787893157,
   'amount1': 3251642992}}}